In [1]:
from operator import itemgetter

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.vectorstores import Qdrant
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.schema.runnable import RunnableSequence

from langchain_setup import pprint_documents, tracing_v2_enabled_if_api_key_set

# 作文抽取
抽取出跟審查中的作文相似的作文。

可以想像網路中可以抄襲的作文有千千萬萬篇，要從中先快速找出相似的作文再來細細比較。這邊為了教學方邊假設只有三篇作文提供比對

In [2]:
# ======================================================
# 資料讀取: 讀取文字檔案成文件 (Documents)
# ======================================================


documents: list[Document] = DirectoryLoader(
    path="./homework/", loader_cls=TextLoader
).load()

pprint_documents(documents)

Document 1:

愛迪生說：「友誼能增進快樂，減少痛苦」，可見朋友有多重要。朋友像一根拐杖，當我跌倒的時候，扶我起來，支撐我繼續往前走；朋友像一位溫柔的老師，當我功課不會寫時，他會很用心的教導我；朋友像一顆開心果，當我難過或傷心時，講笑話給我聽；朋友像一個太陽，當我遇到困難時，覺得人生黑暗的時候，他會溫暖我的心。猶記一年級的時候，我有一位很要好的朋友，她喜歡運動和打球，而我也很喜歡運動，下課的時候，我常常跟她一起玩。她很大方，也很有禮貌，又喜歡幫老師做事，人緣好，所以大家都選她當班長。在她身上我看到很多優點，也學到很多，有了她的陪伴，所以每天在學校都過得很開心。有一次我在操場上跌倒流血，倒在地上爬不起來，是她扶我起來，陪我到健康中心。接下來那幾天，她幫我打掃、交作業，這讓我深深體會好朋友是多麼重要。雖然我們升上中年級後沒有在同一班，但我們還是思念著對方，下課時常常聚在一起，我的心中仍將她視為最好的朋友。如果一個人沒有朋友，就找不到陪她一起玩樂和訴苦的人，這樣就會很孤單。朋友是一生中絕對不能少的，缺少了他，我可能每天心情不好而感到孤獨寂寞。有了好朋友，是一件很幸福的事，也要懂得珍惜，建立的友誼才能長久。

Metadata:{'source': 'homework/1063058_王芸芸.txt'}
----------------------------------------------------------------------------------------------------
Document 2:

如果交了一個朋友，卻不好好珍惜她，很快就會失去了。人如果沒朋友，就有如地球沒有陽光一樣。與朋友相處時，要多微笑，不要每天都愁眉苦，他會以為是自己做了什麼事而讓你生氣，或是不敢靠近你，所以每天都要面帶微笑，身邊自然會聚集一群好朋友。我們平時就要珍惜朋友，就有如英國作家柯爾頓：「真正的友誼猶如健康，只有失去時，才會意識到它的價值。」很多人都是因為失去了友誼，才知道它真正的價值，但往往後悔已經來不及了。朋友之間真的要互相好好珍惜，友誼可以是人生的調味品，也可以是人生的止痛藥。和朋友交往，要多欣賞他的優點，並且隨時注意修正自己的缺點，這樣建立的友誼才能長久。交了一個好朋友不但可以幫助我們，也可以改變我們，讓自己的逐漸成為大家喜歡的人。

Metad

In [3]:
# ======================================================
# 文檔切分：將過長的文檔再細分出更小的文檔
# ======================================================

# 假設我們預算有限，怕整份作文丟給語言模型會花太多錢，我們就會需要把一個長的作文分割成多個段落
# 注意這邊為了教學方便用的是最基礎的以字元 (character) 數來切割，但真實應用中通常會用 `from_tiktoken_encoder`，以 token 數來切割

from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="，",
    chunk_size=500,
    chunk_overlap=30,
)
split_documents: list[Document] = text_splitter.split_documents(documents)

pprint_documents(split_documents)

Document 1:

愛迪生說：「友誼能增進快樂，減少痛苦」，可見朋友有多重要。朋友像一根拐杖，當我跌倒的時候，扶我起來，支撐我繼續往前走；朋友像一位溫柔的老師，當我功課不會寫時，他會很用心的教導我；朋友像一顆開心果，當我難過或傷心時，講笑話給我聽；朋友像一個太陽，當我遇到困難時，覺得人生黑暗的時候，他會溫暖我的心。猶記一年級的時候，我有一位很要好的朋友，她喜歡運動和打球，而我也很喜歡運動，下課的時候，我常常跟她一起玩。她很大方，也很有禮貌，又喜歡幫老師做事，人緣好，所以大家都選她當班長。在她身上我看到很多優點，也學到很多，有了她的陪伴，所以每天在學校都過得很開心。有一次我在操場上跌倒流血，倒在地上爬不起來，是她扶我起來，陪我到健康中心。接下來那幾天，她幫我打掃、交作業，這讓我深深體會好朋友是多麼重要。雖然我們升上中年級後沒有在同一班，但我們還是思念著對方，下課時常常聚在一起，我的心中仍將她視為最好的朋友。如果一個人沒有朋友，就找不到陪她一起玩樂和訴苦的人，這樣就會很孤單。朋友是一生中絕對不能少的，缺少了他，我可能每天心情不好而感到孤獨寂寞。有了好朋友，是一件很幸福的事，也要懂得珍惜，建立的友誼才能長久。

Metadata:{'source': 'homework/1063058_王芸芸.txt'}
----------------------------------------------------------------------------------------------------
Document 2:

如果交了一個朋友，卻不好好珍惜她，很快就會失去了。人如果沒朋友，就有如地球沒有陽光一樣。與朋友相處時，要多微笑，不要每天都愁眉苦，他會以為是自己做了什麼事而讓你生氣，或是不敢靠近你，所以每天都要面帶微笑，身邊自然會聚集一群好朋友。我們平時就要珍惜朋友，就有如英國作家柯爾頓：「真正的友誼猶如健康，只有失去時，才會意識到它的價值。」很多人都是因為失去了友誼，才知道它真正的價值，但往往後悔已經來不及了。朋友之間真的要互相好好珍惜，友誼可以是人生的調味品，也可以是人生的止痛藥。和朋友交往，要多欣賞他的優點，並且隨時注意修正自己的缺點，這樣建立的友誼才能長久。交了一個好朋友不但可以幫助我們，也可以改變我們，讓自己的逐漸成為大家喜歡的人。

Metad

In [4]:
# 建立 Vector store 和 Retriever

## 建立 Vector store
vectore_store = Qdrant.from_documents(
    documents,
    embedding=OpenAIEmbeddings(),
    location=":memory:",
)

## 建立 Retriever
## 篩選出相似度大於閥值 `score_threshold` 且相似度排名前 `k` 高的文件
retriever = vectore_store.as_retriever(search_kwargs=dict(score_threshold=0.7, k=2))

In [5]:
# 建立文件抽取的 runnable

retrieve: RunnableSequence = (
    itemgetter("article") | retriever
)  # {'article': str} -> list[Document]

In [6]:
retrieve.invoke({"article": "哈囉你好嗎？衷心感謝～珍重再見，期待再相逢。"})

[Document(page_content='如果交了一個朋友，卻不好好珍惜她，很快就會失去了。人如果沒朋友，就有如地球沒有陽光一樣。與朋友相處時，要多微笑，不要每天都愁眉苦，他會以為是自己做了什麼事而讓你生氣，或是不敢靠近你，所以每天都要面帶微笑，身邊自然會聚集一群好朋友。我們平時就要珍惜朋友，就有如英國作家柯爾頓：「真正的友誼猶如健康，只有失去時，才會意識到它的價值。」很多人都是因為失去了友誼，才知道它真正的價值，但往往後悔已經來不及了。朋友之間真的要互相好好珍惜，友誼可以是人生的調味品，也可以是人生的止痛藥。和朋友交往，要多欣賞他的優點，並且隨時注意修正自己的缺點，這樣建立的友誼才能長久。交了一個好朋友不但可以幫助我們，也可以改變我們，讓自己的逐漸成為大家喜歡的人。', metadata={'source': 'homework/1063059_陳惠惠.txt'}),
 Document(page_content='愛迪生說：「友誼能增進快樂，減少痛苦」，可見朋友有多重要。朋友像一根拐杖，當我跌倒的時候，扶我起來，支撐我繼續往前走；朋友像一位溫柔的老師，當我功課不會寫時，他會很用心的教導我；朋友像一顆開心果，當我難過或傷心時，講笑話給我聽；朋友像一個太陽，當我遇到困難時，覺得人生黑暗的時候，他會溫暖我的心。猶記一年級的時候，我有一位很要好的朋友，她喜歡運動和打球，而我也很喜歡運動，下課的時候，我常常跟她一起玩。她很大方，也很有禮貌，又喜歡幫老師做事，人緣好，所以大家都選她當班長。在她身上我看到很多優點，也學到很多，有了她的陪伴，所以每天在學校都過得很開心。有一次我在操場上跌倒流血，倒在地上爬不起來，是她扶我起來，陪我到健康中心。接下來那幾天，她幫我打掃、交作業，這讓我深深體會好朋友是多麼重要。雖然我們升上中年級後沒有在同一班，但我們還是思念著對方，下課時常常聚在一起，我的心中仍將她視為最好的朋友。如果一個人沒有朋友，就找不到陪她一起玩樂和訴苦的人，這樣就會很孤單。朋友是一生中絕對不能少的，缺少了他，我可能每天心情不好而感到孤獨寂寞。有了好朋友，是一件很幸福的事，也要懂得珍惜，建立的友誼才能長久。', metadata={'source': 'homework/1063058_王芸芸.txt'})]

# 比對作文


In [7]:
from pprint import pprint

from langchain.prompts import ChatPromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, root_validator
from langchain.output_parsers.openai_functions import PydanticOutputFunctionsParser
from langchain.chains.openai_functions.base import convert_to_openai_function

from langchain_setup import ChatOpenAI

In [8]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位資深的高中國文老師，你可以看出作文之間是否有抄襲。"),
        ("human", "其他的作文：\n{other_articles}"),
        ("human", "審查中的作文如下：\n{article}"),
        ("human", "請老師判斷審查中的作文是否有抄襲其他的作文。"),
    ]
)

example_prompt = prompt.format_messages(
    article="範例作文內容", other_articles="範例其他作文內容一\n範例其他作文內容二"
)
pprint(example_prompt)

[SystemMessage(content='你是一位資深的高中國文老師，你可以看出作文之間是否有抄襲。'),
 HumanMessage(content='其他的作文：\n範例其他作文內容一\n範例其他作文內容二'),
 HumanMessage(content='審查中的作文如下：\n範例作文內容'),
 HumanMessage(content='請老師判斷審查中的作文是否有抄襲其他的作文。')]


In [9]:
class Judement(BaseModel):
    "下達是否抄襲的判定，並附上判定的原因。"

    pirated: bool = Field(description="該審查中的作文是否有抄襲其他作文的行為，包括複製貼上一部份文字、僅更換句子順序等等。")
    reason: str = Field(description="對下達此判定的具體理由。")


function_kwargs = convert_to_openai_function(Judement)
openaifn_kwargs = {
    "functions": [function_kwargs],
    "function_call": {"name": "Judement"},
}

pprint(openaifn_kwargs)

output_parser = PydanticOutputFunctionsParser(pydantic_schema=Judement)

{'function_call': {'name': 'Judement'},
 'functions': [{'description': '下達是否抄襲的判定，並附上判定的原因。',
                'name': 'Judement',
                'parameters': {'description': '下達是否抄襲的判定，並附上判定的原因。',
                               'properties': {'pirated': {'description': '該審查中的作文是否有抄襲其他作文的行為，包括複製貼上一部份文字、僅更換句子順序等等。',
                                                          'title': 'Pirated',
                                                          'type': 'boolean'},
                                              'reason': {'description': '對下達此判定的具體理由。',
                                                         'title': 'Reason',
                                                         'type': 'string'}},
                               'required': ['pirated', 'reason'],
                               'title': 'Judement',
                               'type': 'object'}}]}


In [10]:
judge = prompt | ChatOpenAI(temperature=0, model_name='gpt-4').bind(**openaifn_kwargs) | output_parser

In [11]:
judgement = judge.invoke({"article": "顆顆笑死", "other_articles": "ABCDEFG\nHIJKLMNOP"})
print(judgement)

pirated=False reason='審查中的作文與其他的作文內容完全不同，沒有抄襲的情況。'


# 串接

In [12]:
def combine_documents(documents: list[Document]) -> str:
    text = "\n".join([doc.page_content for doc in documents])
    if not text:
        text = "並無發現其他相似的作文"
    return text

In [13]:
final_runnable = {
    "article": itemgetter("article"),
    "other_articles": retrieve | combine_documents,
} | judge

In [14]:
pirated_article = """朋友像一根拐杖，當我跌倒的時候，扶我起來，支撐我繼續往前走；朋友像一位溫柔的老師，當我功課不會寫時，他會很用心的教導我；朋友像一顆開心果，當我難過或傷心時，講笑話給我聽；朋友像一個太陽，當我遇到困難時，覺得人生黑暗的時候，他會溫暖我的心。愛迪生也說過：「友誼能增進快樂，減少痛苦」，可見朋友有多重要。\n還記得一年級的時候，我有一位很要好的朋友，她喜歡運動和打球，而我也很喜歡運動，下課的時候，我常常跟她一起玩。她很大方，也很有禮貌，又喜歡幫老師做事，人緣好，所以大家都選她當班長。在她身上我看到很多優點，也學到很多，有了她的陪伴，所以每天在學校都過得很開心。有一次我在操場上跌倒流血，倒在地上爬不起來，是她扶我起來，陪我到健康中心。接下來那幾天，她幫我打掃、交作業，這讓我深深體會好朋友是多麼重要。雖然我們升上中年級後沒有在同一班，但我們還是思念著對方，下課時常常聚在一起，我的心中仍將她視為最好的朋友。如果一個人沒有朋友，就找不到陪她一起玩樂和訴苦的人，這樣就會很孤單。朋友是一生中絕對不能少的，缺少了他，我可能每天心情不好而感到孤獨寂寞。有了好朋友，是一件很幸福的事，也要懂得珍惜，建立的友誼才能長久。
"""

not_pirated_article = """剛認識新朋友時，我常說『請』、『謝謝』、『對不起』，來保持我的形象。認識一陣子後，我常說『誒Ｘ等一下要吃什麼啦』。"""

In [15]:
final_runnable.invoke({"article": pirated_article})

Judement(pirated=True, reason='審查中的作文與其他的作文有多處相同的句子，例如「朋友像一根拐杖，當我跌倒的時候，扶我起來，支撐我繼續往前走」、「愛迪生也說過：「友誼能增進快樂，減少痛苦」，可見朋友有多重要」等等，這些都是直接從其他的作文中複製過來的，因此，審查中的作文有抄襲的行為。')

In [16]:
final_runnable.invoke({"article": not_pirated_article})

Judement(pirated=False, reason='審查中的作文與其他的作文內容並無重複或相似之處，且主題與表達方式也有所不同。因此，審查中的作文並無抄襲其他作文的情況。')